In [ ]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from configure import *
import time
from concurrent.futures import ProcessPoolExecutor
import psutil

dirname = os.getcwd()
flag_smoothing=True #是否对导入的电势场格点数据做平滑化，如果True，那么平滑化函数默认按照下文def smoothing(data)
filename=os.path.join(dirname, "../../../data/monolithic20241118.csv") #文件名：导入的电势场格点数据
basis_filename=os.path.join(dirname, "electrode_basis.json")#文件名：自定义Basis设置 #可以理解为一种基矢变换，比如"U1"相当于电势场组合"esbe1"*0.5+"esbe1asy"*-0.5
basis = Data_Loader(filename, basis_filename, flag_smoothing)
basis.loadData()
configure = Configure(basis=basis)
# configure.load_from_file(os.path.join(dirname, "../saves/saved_config_regression_0.01_1000.json"))
configure.load_from_file(os.path.join(dirname, "../saves/flat_28.json"))
Vs = configure.static_potential()
Vpp = configure.pseudo_potential()
Vt = Vs + Vpp

N = 10000

grid_x, grid_y, grid_z = basis.coordinate_um
Vt_itp = RegularGridInterpolator((grid_x, grid_y, grid_z), Vt, method="linear", bounds_error=True)

def Energy(r):
    N = r.shape[0]
    Vt_min = np.min(Vt_itp(r))
    return sum(Vt_itp(r)-Vt_min)

def Hessian_ij(i, j, N, DIM, r1, Er, h=0.1):
    if i == j:
        r_ip = r_im = r1
        r_ip[i] += h
        r_im[i] -= h
        return (Energy(r_ip.reshape(N,DIM))-2*Er+Energy(r_im.reshape(N,DIM)))/h**2
    else:
        r_ip_jp = r_ip_jm = r_im_jp = r_im_jm = r1
        r_ip_jp[i] += h
        r_ip_jp[j] += h
        r_ip_jm[i] += h
        r_ip_jm[j] -= h
        r_im_jp[i] -= h
        r_im_jp[j] += h
        r_im_jm[i] -= h
        r_im_jm[j] -= h
        return i, j, (Energy(r_ip_jp.reshape(N,DIM))-Energy(r_ip_jm.reshape(N,DIM))-Energy(r_im_jp.reshape(N,DIM))+Energy(r_im_jm.reshape(N,DIM)))/(4*h**2)

def Hessian(r, h=0.1):
    N, DIM = r.shape
    H = np.zeros((N*DIM, N*DIM))
    Er = Energy(r)
    r1 = r.ravel()
    with ProcessPoolExecutor() as executor:
        max_workers = executor._max_workers
        print(f"最大并行进程数: {max_workers}")
        Futures = [executor.submit(Hessian_ij, i, j, N, DIM, r1, Er, h) for i in range(N*DIM) for j in range(N*DIM)]
        parent = psutil.Process()  # 当前 Python 进程
        children = parent.children(recursive=True)  # 获取所有子进程
        print(f"当前运行的子进程数 by psutil: {len(children)}")
        time.sleep(0.1)  # 短暂等待以确保子进程启动
        active_processes = len(multiprocessing.active_children())
        print(f"当前活跃进程数 by multiprocessing: {active_processes}")
        for f in Futures:
            i, j, Hij = f.result()
            H[i][j] = Hij
    np.save("../data_cache/%d/Hessian/flat_28_54.432us.npy"%N)
    return H

r = np.load("../data_cache/%d/ion_pos/flat_28_54.432us.npy"%N)
start = time.time()
H = Hessian(r)
end = time.time()
print("Total time spent: ", end-start, "s")

dl: 2.7437758083117576e-07 m
dt: 9.022389064166401e-09 s
dV: 0.001312028248176 V
Loaded custom basis: {'RF': 'esbe', 'U1': 'esbe1', 'U2': 'esbe2', 'U3': 'esbe3', 'U4': 'esbe4', 'U5': 'esbe5', 'U6': 'esbe6', 'U7': 'esbe7'}
加载自定义Basis设置
self.unit_l= 0.001
最大并行进程数: 208


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fcb09b430d0>>
Traceback (most recent call last):
  File "/root/anaconda3/envs/ism_ylc/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(H)

In [24]:
import numpy as np

a = [i+j for i in range(3) for j in range(2)]
print(a)

[0, 1, 1, 2, 2, 3]
